In [50]:
!pip install pyarrow
!pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pickle as pkl

## Importing the model and CSV file

In [52]:
df = pd.read_csv("../troop_movements_1m.csv")
with open ('../model/clf_decision_tree.pkl', 'rb') as f:
    model = pkl.load(f)
df

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
0,2023-06-03 03:19:15,919214,tie_silencer,2.0,5.0,9,3,Aleen Minor
1,2023-02-24 13:50:40,9467154,stormtrooper,9.0,0.0,9,1,Malastare
2,2023-03-29 19:54:55,6585778,tie_silencer,0.0,6.0,5,9,Serenno
3,2023-04-30 00:58:11,3878023,tie_silencer,4.0,2.0,9,9,Tund
4,2023-04-10 22:00:26,5537117,at-st,6.0,8.0,5,8,Skako
...,...,...,...,...,...,...,...,...
999995,2023-05-03 18:48:56,3553240,tie_fighter,8.0,4.0,8,1,Chandrila
999996,2023-05-25 13:45:35,1662636,at-at,0.0,3.0,8,6,Trandosha
999997,2023-05-17 12:16:37,2362262,tie_fighter,6.0,8.0,7,9,Kashyyyk
999998,2023-03-17 08:04:26,7669749,tie_fighter,9.0,3.0,3,5,Zolan


## Cleaning up the data

In [59]:
# Cleaning 
df['unit_type'] = df['unit_type'].replace('invalid_unit', 'unknown')

# Filling in missing values
df['location_x'] = df['location_x'].ffill()
df['location_y'] = df['location_y'].ffill()
df

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld,is_resistance
0,2023-06-03 03:19:15,919214,tie_silencer,2.0,5.0,9,3,Aleen Minor,False
1,2023-02-24 13:50:40,9467154,stormtrooper,9.0,0.0,9,1,Malastare,True
2,2023-03-29 19:54:55,6585778,tie_silencer,0.0,6.0,5,9,Serenno,False
3,2023-04-30 00:58:11,3878023,tie_silencer,4.0,2.0,9,9,Tund,False
4,2023-04-10 22:00:26,5537117,at-st,6.0,8.0,5,8,Skako,False
...,...,...,...,...,...,...,...,...,...
999995,2023-05-03 18:48:56,3553240,tie_fighter,8.0,4.0,8,1,Chandrila,True
999996,2023-05-25 13:45:35,1662636,at-at,0.0,3.0,8,6,Trandosha,False
999997,2023-05-17 12:16:37,2362262,tie_fighter,6.0,8.0,7,9,Kashyyyk,True
999998,2023-03-17 08:04:26,7669749,tie_fighter,9.0,3.0,3,5,Zolan,True


## Saving to Parquet file

In [54]:
df.to_parquet('../parquet/troops_movements_1m.parquet')

## 

## Loading data from Parquet file

In [55]:
filepath = '../parquet/troops_movements_1m.parquet'
df2 = pd.read_parquet(filepath)
df2

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
0,2023-06-03 03:19:15,919214,tie_silencer,2.0,5.0,9,3,Aleen Minor
1,2023-02-24 13:50:40,9467154,stormtrooper,9.0,0.0,9,1,Malastare
2,2023-03-29 19:54:55,6585778,tie_silencer,0.0,6.0,5,9,Serenno
3,2023-04-30 00:58:11,3878023,tie_silencer,4.0,2.0,9,9,Tund
4,2023-04-10 22:00:26,5537117,at-st,6.0,8.0,5,8,Skako
...,...,...,...,...,...,...,...,...
999995,2023-05-03 18:48:56,3553240,tie_fighter,8.0,4.0,8,1,Chandrila
999996,2023-05-25 13:45:35,1662636,at-at,0.0,3.0,8,6,Trandosha
999997,2023-05-17 12:16:37,2362262,tie_fighter,6.0,8.0,7,9,Kashyyyk
999998,2023-03-17 08:04:26,7669749,tie_fighter,9.0,3.0,3,5,Zolan


In [58]:
features = ['homeworld', 'unit_type']
X = df2[features]
X = pd.get_dummies(X)
X.columns

model.predict(X)
df['is_resistance'] = model.predict(X)
df

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld,is_resistance
0,2023-06-03 03:19:15,919214,tie_silencer,2.0,5.0,9,3,Aleen Minor,False
1,2023-02-24 13:50:40,9467154,stormtrooper,9.0,0.0,9,1,Malastare,True
2,2023-03-29 19:54:55,6585778,tie_silencer,0.0,6.0,5,9,Serenno,False
3,2023-04-30 00:58:11,3878023,tie_silencer,4.0,2.0,9,9,Tund,False
4,2023-04-10 22:00:26,5537117,at-st,6.0,8.0,5,8,Skako,False
...,...,...,...,...,...,...,...,...,...
999995,2023-05-03 18:48:56,3553240,tie_fighter,8.0,4.0,8,1,Chandrila,True
999996,2023-05-25 13:45:35,1662636,at-at,0.0,3.0,8,6,Trandosha,False
999997,2023-05-17 12:16:37,2362262,tie_fighter,6.0,8.0,7,9,Kashyyyk,True
999998,2023-03-17 08:04:26,7669749,tie_fighter,9.0,3.0,3,5,Zolan,True
